In [39]:
import pandas as pd
import os
import openpyxl           # pip install openpyxl

In [40]:
merge_RD = pd.read_csv('./merge_RD.csv')
merge_info = pd.read_csv('./merge_info.csv')
merge_patent = pd.read_csv('./merge_patent.csv')

C:\Users\DMC CONET\AppData\Local\Temp\ipykernel_9944\3339999625.py:1: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  merge_RD = pd.read_csv('./merge_RD.csv')


# 특허지수 계산하기

In [51]:
PATENT = pd.read_csv('./PATENT.csv')
INFO_RD = pd.read_csv('./INFO_RD.csv')
code = pd.read_csv('./code.csv')

In [67]:
# PATENT[['사업자등록번호','출원번호','특허명']]
data_pivot = pd.pivot_table(PATENT, index='사업자등록번호', values = '순번', aggfunc='count')
count = data_pivot.순번.tolist()

In [102]:
# code1 = code[['대분류코드','대분류항목', '세세분류코드']]
# df1 = INFO_RD[['사업자등록번호','산업분류코드']]
code1.info()
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1196 entries, 0 to 1196
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   대분류코드   1196 non-null   object
 1   대분류항목   1196 non-null   object
 2   세세분류코드  1196 non-null   int32 
dtypes: int32(1), object(2)
memory usage: 65.0+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120416 entries, 0 to 120415
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   사업자등록번호  120416 non-null  int64
 1   산업분류코드   120416 non-null  int32
dtypes: int32(1), int64(1)
memory usage: 1.4 MB


In [101]:
code1 = code1.astype({'세세분류코드':'int'})
df1 = df1.astype({'산업분류코드':'int'})

In [105]:
df_merge = pd.merge(code1, df1, left_on='세세분류코드', right_on='산업분류코드', how='right')

In [108]:
df_merge.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=False)
df_merge = df_merge.dropna()

In [113]:
df_merge = df_merge[['대분류코드','사업자등록번호']]

In [128]:
result = pd.merge(PATENT, df_merge, left_on='사업자등록번호', right_on='사업자등록번호', how='left')
result = result[['대분류코드','사업자등록번호', '특허명칭', 'IPC코드내용', '출원인명', 'NDSL상세보기URL', '출원번호',
       '특허명', '출원날짜', 'CPC번호', '인용수', '피인용수', '발명자수', '청구항수']]
# 대분류코드가 null인 행만 drop
result = result.dropna(subset=['대분류코드'])
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40331 entries, 0 to 67219
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   대분류코드        40331 non-null  object 
 1   사업자등록번호      40331 non-null  int64  
 2   특허명칭         36971 non-null  object 
 3   IPC코드내용      29160 non-null  object 
 4   출원인명         36971 non-null  object 
 5   NDSL상세보기URL  36971 non-null  object 
 6   출원번호         40327 non-null  float64
 7   특허명          40327 non-null  object 
 8   출원날짜         40327 non-null  float64
 9   CPC번호        39178 non-null  object 
 10  인용수          13741 non-null  float64
 11  피인용수         8218 non-null   float64
 12  발명자수         40327 non-null  float64
 13  청구항수         40125 non-null  float64
dtypes: float64(6), int64(1), object(7)
memory usage: 4.6+ MB


In [130]:
result.head(1)

,대분류코드,사업자등록번호,특허명칭,IPC코드내용,출원인명,NDSL상세보기URL,출원번호,특허명,출원날짜,CPC번호,인용수,피인용수,발명자수,청구항수
0,G,1240400001,비제로 계수들의 위치를 표현하는 방법 및 장치,H04N-019/129,에스케이텔레콤 주***,http://click.ndsl.kr/servlet/OpenAPIDetailView...,1.020180e+12,단말장치 및 그 장치의 동작 방법,20180711.0,H04M1/724,NaN,NaN,2.0,10.0


In [ ]:
# R&D) 전체 특허출원 수 ==> 40331 
# result['stone'] = 0

In [156]:
# 특정기술분야에서의 특허출원 수
# 분야별 특허 count
pv1 = pd.pivot_table(result, index='대분류코드', values = 'stone', aggfunc='count')
pv1.reset_index(inplace=True)
pv1.rename(columns = {"stone": "분야별특허수"}, inplace = True)
pv1

,대분류코드,분야별특허수
0,A,2
1,C,13959
2,F,8806
3,G,7878
4,H,100
5,J,9184
6,N,300
7,P,1
8,Q,100
9,S,1


In [175]:
# 특정기술분야에서 특정출원국의 특허출원 수
# result['stone'] = 0
pv2 = pd.pivot_table(result, index=['대분류코드','사업자등록번호'], values = 'stone', aggfunc='count')
# pv2.reset_index(inplace=True)
# pv2.rename(columns = {"stone": "사업자별특허수"}, inplace = True)
pv2

stone
대분류코드 사업자등록번호          
A     2158600382      2
C     1010600385   1000
      1058603820    576
      1058607750     90
      1068108761   1000
...                 ...
J     6218104310   1000
N     1178109457    300
P     6178200261      1
Q     1051607933    100
S     2118200751      1

[121 rows x 1 columns]

In [158]:
# 특정출원국의 전체 특허출원수
pv3 = pd.pivot_table(result, index='사업자등록번호', values = 'stone', aggfunc='count')
pv3.reset_index(inplace=True)
pv3.rename(columns = {"stone": "사업자별특허수"}, inplace = True)
pv3

,사업자등록번호,사업자별특허수
0,1010600385,1000
1,1018100111,240
2,1051607933,100
3,1051807896,100
4,1058603820,576
...,...,...
116,6915103608,20
117,8078102763,500
118,8128803909,900
119,8148800219,10


In [140]:
result.columns
result1 = result[['대분류코드', '사업자등록번호', '출원번호', '특허명', '인용수', '피인용수']]

Index(['대분류코드', '사업자등록번호', '특허명칭', 'IPC코드내용', '출원인명', 'NDSL상세보기URL', '출원번호',
       '특허명', '출원날짜', 'CPC번호', '인용수', '피인용수', '발명자수', '청구항수', 'stone'],
      dtype='object')

In [169]:
m1 = pd.merge(result1, pv1, left_on='대분류코드', right_on='대분류코드', how='left')
m2 = pd.merge(m1, pv3, left_on='사업자등록번호', right_on='사업자등록번호', how='left')
m2

,대분류코드,사업자등록번호,출원번호,특허명,인용수,피인용수,분야별특허수,사업자별특허수
0,G,1240400001,1.020180e+12,단말장치 및 그 장치의 동작 방법,NaN,NaN,7878,100
1,G,1240400001,1.020180e+12,데이터 전송경로 스위칭 방법,1.0,NaN,7878,100
2,G,1240400001,1.020180e+12,기지국장치 및 그 장치의 동작 방법,NaN,NaN,7878,100
3,G,1240400001,1.020180e+12,영상 투사 장치,1.0,NaN,7878,100
4,G,1240400001,1.020180e+12,Cnn 기반의 영상 부호화 또는 복호화 장치 및 방법,NaN,NaN,7878,100
...,...,...,...,...,...,...,...,...
40326,F,8398709840,1.020160e+12,플로트의 볼트체결구조,1.0,1.0,8806,1000
40327,F,8398709840,1.020160e+12,플로트의 볼트체결구조,1.0,1.0,8806,1000
40328,F,8398709840,1.020160e+12,플로트의 볼트체결구조,1.0,1.0,8806,1000
40329,F,8398709840,1.020160e+12,플로트의 볼트체결구조,1.0,1.0,8806,1000


In [181]:
m2[m2.사업자등록번호 == 1240400001].대분류코드 

,대분류코드,사업자등록번호,출원번호,특허명,인용수,피인용수,분야별특허수,사업자별특허수,사업자/분야,AI
0,G,1240400001,1.020180e+12,단말장치 및 그 장치의 동작 방법,NaN,NaN,7878,100,0.012694,0.001083
1,G,1240400001,1.020180e+12,데이터 전송경로 스위칭 방법,1.0,NaN,7878,100,0.012694,0.001083
2,G,1240400001,1.020180e+12,기지국장치 및 그 장치의 동작 방법,NaN,NaN,7878,100,0.012694,0.001083
3,G,1240400001,1.020180e+12,영상 투사 장치,1.0,NaN,7878,100,0.012694,0.001083
4,G,1240400001,1.020180e+12,Cnn 기반의 영상 부호화 또는 복호화 장치 및 방법,NaN,NaN,7878,100,0.012694,0.001083
...,...,...,...,...,...,...,...,...,...,...
95,G,1240400001,1.020180e+12,패킷 데이터 네트워크 게이트웨이 장치와 그 호 처리 방법,1.0,NaN,7878,100,0.012694,0.001083
96,G,1240400001,1.020180e+12,Rf 중계기 및 그 제어방법,1.0,NaN,7878,100,0.012694,0.001083
97,G,1240400001,1.020180e+12,클라우드 컴퓨팅 기반의 모바일 오피스를 위한 보안 문서 처리 방법,2.0,2.0,7878,100,0.012694,0.001083
98,G,1240400001,1.020180e+12,영상 투사 장치,1.0,NaN,7878,100,0.012694,0.001083


In [170]:
m2['사업자/분야'] = m2.사업자별특허수 / m2.분야별특허수
m2

,대분류코드,사업자등록번호,출원번호,특허명,인용수,피인용수,분야별특허수,사업자별특허수,사업자/분야
0,G,1240400001,1.020180e+12,단말장치 및 그 장치의 동작 방법,NaN,NaN,7878,100,0.012694
1,G,1240400001,1.020180e+12,데이터 전송경로 스위칭 방법,1.0,NaN,7878,100,0.012694
2,G,1240400001,1.020180e+12,기지국장치 및 그 장치의 동작 방법,NaN,NaN,7878,100,0.012694
3,G,1240400001,1.020180e+12,영상 투사 장치,1.0,NaN,7878,100,0.012694
4,G,1240400001,1.020180e+12,Cnn 기반의 영상 부호화 또는 복호화 장치 및 방법,NaN,NaN,7878,100,0.012694
...,...,...,...,...,...,...,...,...,...
40326,F,8398709840,1.020160e+12,플로트의 볼트체결구조,1.0,1.0,8806,1000,0.113559
40327,F,8398709840,1.020160e+12,플로트의 볼트체결구조,1.0,1.0,8806,1000,0.113559
40328,F,8398709840,1.020160e+12,플로트의 볼트체결구조,1.0,1.0,8806,1000,0.113559
40329,F,8398709840,1.020160e+12,플로트의 볼트체결구조,1.0,1.0,8806,1000,0.113559


In [182]:
m2['AI'] = m2['사업자별특허수']*67220 / (m2.사업자별특허수 * m2.분야별특허수)
m2

,대분류코드,사업자등록번호,출원번호,특허명,인용수,피인용수,분야별특허수,사업자별특허수,사업자/분야,AI
0,G,1240400001,1.020180e+12,단말장치 및 그 장치의 동작 방법,NaN,NaN,7878,100,0.012694,8.532622
1,G,1240400001,1.020180e+12,데이터 전송경로 스위칭 방법,1.0,NaN,7878,100,0.012694,8.532622
2,G,1240400001,1.020180e+12,기지국장치 및 그 장치의 동작 방법,NaN,NaN,7878,100,0.012694,8.532622
3,G,1240400001,1.020180e+12,영상 투사 장치,1.0,NaN,7878,100,0.012694,8.532622
4,G,1240400001,1.020180e+12,Cnn 기반의 영상 부호화 또는 복호화 장치 및 방법,NaN,NaN,7878,100,0.012694,8.532622
...,...,...,...,...,...,...,...,...,...,...
40326,F,8398709840,1.020160e+12,플로트의 볼트체결구조,1.0,1.0,8806,1000,0.113559,7.633432
40327,F,8398709840,1.020160e+12,플로트의 볼트체결구조,1.0,1.0,8806,1000,0.113559,7.633432
40328,F,8398709840,1.020160e+12,플로트의 볼트체결구조,1.0,1.0,8806,1000,0.113559,7.633432
40329,F,8398709840,1.020160e+12,플로트의 볼트체결구조,1.0,1.0,8806,1000,0.113559,7.633432
